In [1]:
import torch
import torchvision
import torchvision.transforms as tr
from torch.utils.data import DataLoader

# Load Data set

In [2]:
transform = tr.Compose(
    [tr.ToTensor(),
     tr.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5))])

# 파이토치에서 제공하는 CIFAR10
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = DataLoader(trainset, batch_size=8, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = DataLoader(testset, batch_size=8, shuffle=False, num_workers=2)

# 10가지 클래스가 존재
classes = ('plane', 'car','bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

100%|██████████| 170498071/170498071 [00:02<00:00, 77937991.36it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


# Build a model

In [3]:
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module): # 우선 nn.Module을 상속 받기
  def __init__(self):
    super(Net, self).__init__()
    self.conv1 = nn.Conv2d(3, 6, 5) # 들어온 채널, output채널, kernel_size
    self.pool = nn.MaxPool2d(2, 2)
    self.conv2 = nn.Conv2d(6, 16, 5)
    self.fc1 = nn.Linear(16*5*5, 120)
    self.fc2 = nn.Linear(120, 84)
    self.fc3 = nn.Linear(84, 10)

  def forward(self, x):
    x = self.pool(F.relu(self.conv1(x)))
    x = self.pool(F.relu(self.conv2(x)))
    x = x.view(-1, 16*5*5)
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = self.fc3(x)
    return x

# 클래스에 대한 인스턴스 생성
net = Net()

In [4]:
print(net)
#

Net(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


# Implement the model with training set

In [5]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9) # net안의 파라미터들을 업데이트 시켜준다.
# lr은 scheduling을 통해 가변적으로 운영 가능

In [35]:
# 실질적인 학습 부분
for epoch in range(1): # for문을 다 돌면 데이터를 한 번 다 본 것인데, 몇 번 반복해서 보게 하려고
  running_loss = 0.0
  for i, data in enumerate(trainloader, 0): # 8개씩 6250개가 들어온다. 즉, 한 번에 8개씩 loss구하고 학습
    inputs, labels = data

    #옵티마이저 초기화 작업
    optimizer.zero_grad()

    # 모델에 넣기
    outputs = net(inputs)
    # 실제 label하고 loss 계산
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step() # optimizer 계산

    # print statistics
    running_loss += loss.item() # scalar로 바꿔서 더하기
    if i % 2000 == 1999 :
      print('[%d, %5d] loss: %.3f' %
            (epoch + 1, i + 1, running_loss/2000))
      running_loss = 0.0 # 초기화

print('Finished Training')

# ++ loss가 가장 작을 때 모델을 저장하게 할 수도 있다! + gpu연산을 하게 할 수도 있다!

[1,  2000] loss: 2.250
[1,  4000] loss: 1.828
[1,  6000] loss: 1.626
Finished Training


# Save the trained model

In [36]:
PATH = './cifar_net.pth'
torch.save(net.state_dict(), PATH)

# Load the pre-trained model

In [37]:
net = Net()
net.load_state_dict(torch.load(PATH))

<All keys matched successfully>

* output은 미니배치의 결과가 산출되기 때문에 for문을 통해 test 전체의 예측값을 구해야 한다.

In [55]:
correct = 0 # 몇 개 맞췄는지
total = 0 # 총 이미지 개수 10000개
# 추적을 안 한다.
with torch.no_grad():
  for data in testloader:
    images, labels = data
    outputs = net(images) # 10개에 대한 각 값
    _, predicted = torch.max(outputs.data, 1) # 8개의 이미지 각각 클래스 중 가장 큰 것을 찾음 : class 1개 선택
    total += labels.size(0)
    correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images : %d %%' % (
    correct / total * 100))

Accuracy of the network on the 10000 test images : 41 %


In [46]:
for i, data in enumerate(testloader):
  if i == 1:
    break
  images, labels = data
  print(len(images), labels.size())

8 torch.Size([8])


In [58]:
torch.max(outputs.data, 1)

torch.return_types.max(
values=tensor([2.7734, 3.2942, 2.1893, 1.2707, 2.4127, 3.4155, 3.1370, 1.5827]),
indices=tensor([3, 3, 6, 0, 3, 5, 1, 7]))

In [61]:
(labels == torch.max(outputs.data, 1)[1]).sum().item()

5